In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yaml # If you don't have this package use 'pip install pyyaml' into commandline or terminal
import seaborn as sns
# Matplotlib Config
#%matplotlib inline
#plt.style.use('fivethirtyeight')
# Turn 'building.yaml' file into a python dictionary using PyYAML 
with open('Files/building.yaml') as f:
    building_data = f.read()
    building_data = yaml.load(building_data)
    f.close()
building = pd.DataFrame(building_data) #building contains only two rows out of which only one(first one) contains actual info
building = pd.DataFrame(building['building1'][0])
    
### Column Names for our dataframe
col_names = ['sec', 
             'agent_id',
             'agent_type',
             'has_luggage',
             'is_disabled',
             'X', 
             'Y',
             'Z',
             'velocity',
             'queue_id',
             'lookUp_X',
             'lookUp_Y']
df1 = pd.read_csv('Files/Agents/thread_0.txt', header=0, names=col_names)
df2 = pd.read_csv('Files/Agents/thread_1.txt', header=0, names=col_names)
df3 = pd.read_csv('Files/Agents/thread_2.txt', header=0, names=col_names)
df4 = pd.read_csv('Files/Agents/thread_3.txt', header=0, names=col_names)
df5 = pd.read_csv('Files/Agents/thread_4.txt', header=0, names=col_names)
df6 = pd.read_csv('Files/Agents/thread_5.txt', header=0, names=col_names)
df7 = pd.read_csv('Files/Agents/thread_6.txt', header=0, names=col_names)
df8 = pd.read_csv('Files/Agents/thread_7.txt', header=0, names=col_names)

frames = [df1,df2,df3,df4,df5,df6,df7,df8]
df = pd.concat(frames)

In [2]:
df_test = df[df['Z'] >= 0]

In [3]:
df_test['binned_X'] = pd.qcut(df_test['X'],20)
df_test['binned_Y'] = pd.qcut(df_test['Y'],20)

C:\Users\Saifi\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\Saifi\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [92]:
# Generate Images for each T
count = 0
for t in df_test['sec'].unique():
    df_test_group = df_test[df_test['sec'] == t].groupby(['binned_X','binned_Y'],as_index=False).count()
    df_test_group = df_test_group[['binned_X','binned_Y','agent_id']]
    count += 1
    
    plt.figure(figsize=(10,10))
    img = sns.heatmap(df_test_group.pivot(index='binned_Y',columns='binned_X',values='agent_id'),linewidths=0.2,vmin=0,vmax=250)
    plt.tight_layout()
    img.figure.text(0.05, 0.1, "t = " + str(count), ha ='left', fontsize = 15)
    img.get_figure().savefig('heatmap/heatmap' + str(count) + '.png')
    plt.close()

In [93]:
# Create GIF

import imageio

filenames = ['heatmap/heatmap' + str(x) + '.png' for x in range(1,1200)]

images = []

for filename in filenames:
    images.append(imageio.imread(filename))

imageio.mimsave('heatmap_by_time.gif', images)